In [1]:
from elasticsearch_dsl import connections, Index, Search
from elasticsearch_dsl.query import Range, Terms, Term
from elasticsearch_dsl.aggs import Terms as PerTerms, Cardinality
from json import loads, load
from pandas import DataFrame, NA
from pathlib import Path
from favicon import get as get_favicon
from requests import get

/tmp/ipykernel_2649821/2977042073.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  from pandas import DataFrame, NA


In [2]:
ELASTIC_HOST = "https://elasticsearch.srv.webis.de"
ELASTIC_PORT = 9200
ELASTIC_USER = "ajjxp"
# As a way to hide the password at least from the notebook, enter a path to a file here, which only contains the password for Elastic.
ELASTIC_PASSWORD_FILE = Path("~/.local/share/passwords/webis-elasticsearch.txt").expanduser()
INDEX = "corpus_mastodon_statuses*"

INSTANCE_DATA_PATH = Path("/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-ernst/fedi_data/2024-01-30/05.jsonl")
INSTANCES_PATH = Path("/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-ernst/sample/04/instances.txt")
REMOVED_INSTANCES_PATH = Path("/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-ernst/sample/04/instances_removed_for_crawling_errors.json")

OUTPUT_PATH = Path("../data")

NUM_TOP_INSTANCES = 10

# Limit the Elastic searches to a specific date range. Crawling started on 2023-12-21.
DATE_AFTER = "2023-12-01T00:00:00"
## Ca. "2024-01-30T12:00:00" is the time when a new version of the fediverse data was gahtered.
DATE_BEFORE = "2024-02-22T00:00:00"

In [3]:
# Connect to Elastic.
with ELASTIC_PASSWORD_FILE.open("r") as f:
    password = f.readline().strip("\n")
elastic = connections.create_connection(
    hosts=ELASTIC_HOST + ":" + str(ELASTIC_PORT),
    basic_auth=(ELASTIC_USER, password),
    timeout=300
)

In [4]:
# Prepare the date query
date_query = Range(crawled_at={"gte": DATE_AFTER, "lte": DATE_BEFORE, "format" : "date_hour_minute_second"})
date_query.to_dict()

{'range': {'crawled_at': {'gte': '2023-12-01T00:00:00',
   'lte': '2024-02-22T00:00:00',
   'format': 'date_hour_minute_second'}}}

In [5]:
base_search: Search = Index(INDEX)\
    .search()\
    .filter(date_query)
base_search.count()

733422534

In [6]:
crawled_instances_count_search: Search = base_search.params(size=0)
crawled_instances_count_search.aggs\
    .metric(
        "instances", 
        Cardinality(field="crawled_from_instance.keyword")
    )
crawled_instances_count = crawled_instances_count_search.execute()\
    .aggs.instances.value
crawled_instances_count

1015

In [7]:
crawled_counts_search: Search = base_search.params(size=0)
crawled_counts_search.aggs\
    .bucket(
        "instances",
        PerTerms(field="crawled_from_instance.keyword", size=crawled_instances_count)
    )
    
crawled_counts_buckets = crawled_counts_search.execute().aggs.instances.buckets
crawled_counts = {
    bucket.key: bucket.doc_count
    for bucket in crawled_counts_buckets
}
crawled_counts

{'mastodon.social': 15668157,
 'mastodon.online': 9857093,
 'mstdn.social': 9638803,
 'ohai.social': 8754882,
 'mas.to': 8208704,
 'mastodon.world': 8149459,
 'universeodon.com': 7489453,
 'social.vivaldi.net': 7194763,
 'techhub.social': 7121898,
 'toot.community': 6660323,
 'convo.casa': 6446080,
 'mastodonapp.uk': 6324971,
 'flipboard.social': 6310298,
 'cupoftea.social': 5642647,
 'masto.ai': 5383642,
 'mastodon.sdf.org': 5241825,
 'hachyderm.io': 5167978,
 'fosstodon.org': 5140898,
 'vmst.io': 5140272,
 'infosec.exchange': 4950838,
 'fedibird.com': 4936551,
 'veganism.social': 4882563,
 'c.im': 4507321,
 'qoto.org': 4506541,
 'mstdn.party': 4430716,
 'mastodon.org.uk': 4395694,
 'chaosfem.tw': 4342564,
 'troet.cafe': 4271040,
 'sfba.social': 4035500,
 'mstdn.nijist.info': 3874928,
 'mstdn.business': 3801452,
 'chaos.social': 3751356,
 'mindly.social': 3746077,
 'mastodon.tokyo': 3643814,
 'fairy.id': 3629844,
 'expressional.social': 3613162,
 'aus.social': 3593440,
 'mastodon.scot

In [8]:
crawled_federated_counts_search: Search = base_search.params(size=0)\
    .filter(Term(is_local=False))
crawled_federated_counts_search.aggs\
    .bucket(
        "instances",
        PerTerms(field="crawled_from_instance.keyword", size=crawled_instances_count)
    )
    
crawled_federated_counts_buckets = crawled_federated_counts_search.execute().aggs.instances.buckets
crawled_federated_counts = {
    bucket.key: bucket.doc_count
    for bucket in crawled_federated_counts_buckets
}
crawled_federated_counts

{'mastodon.social': 12542364,
 'mastodon.online': 9570010,
 'mstdn.social': 9267685,
 'ohai.social': 8735250,
 'mas.to': 8034084,
 'mastodon.world': 7968251,
 'universeodon.com': 7399674,
 'social.vivaldi.net': 7055731,
 'techhub.social': 7047603,
 'toot.community': 6630708,
 'convo.casa': 6444904,
 'flipboard.social': 6291206,
 'mastodonapp.uk': 6253585,
 'cupoftea.social': 5631250,
 'masto.ai': 5309531,
 'mastodon.sdf.org': 5211530,
 'vmst.io': 5134492,
 'hachyderm.io': 5109150,
 'fosstodon.org': 5095085,
 'infosec.exchange': 4878992,
 'veganism.social': 4878227,
 'fedibird.com': 4677200,
 'qoto.org': 4492760,
 'c.im': 4472183,
 'mstdn.party': 4415015,
 'mastodon.org.uk': 4392730,
 'chaosfem.tw': 4326409,
 'troet.cafe': 4197062,
 'sfba.social': 3990118,
 'mstdn.nijist.info': 3874627,
 'mstdn.business': 3796904,
 'mindly.social': 3730950,
 'chaos.social': 3686665,
 'mastodon.tokyo': 3643534,
 'fairy.id': 3627017,
 'expressional.social': 3603820,
 'aus.social': 3557073,
 'mastodon.scot

In [9]:
crawled_local_counts_search: Search = base_search.params(size=0)\
    .filter(Term(is_local=True))
crawled_local_counts_search.aggs\
    .bucket(
        "instances",
        PerTerms(field="crawled_from_instance.keyword", size=crawled_instances_count)
    )
    
crawled_local_counts_buckets = crawled_local_counts_search.execute().aggs.instances.buckets
crawled_local_counts = {
    bucket.key: bucket.doc_count
    for bucket in crawled_local_counts_buckets
}
crawled_local_counts

{'mastodon.social': 3125793,
 'mstdn.social': 371118,
 'masto.globaleas.org': 305767,
 'mastodon.online': 287083,
 'fedibird.com': 259351,
 'rss-mstdn.studiofreesia.com': 202735,
 'botsin.space': 184529,
 'mastodon.world': 181208,
 'mas.to': 174620,
 'wxw.moe': 142162,
 'social.vivaldi.net': 139032,
 'mastodon.nl': 137952,
 'imastodon.net': 107952,
 'pravda.me': 107502,
 'mstdn.kemono-friends.info': 104249,
 'universeodon.com': 89779,
 'piaille.fr': 83019,
 'pokemon.mastportal.info': 80766,
 'techhub.social': 74295,
 'masto.ai': 74111,
 'troet.cafe': 73978,
 'infosec.exchange': 71846,
 'mastodonapp.uk': 71386,
 'vocalodon.net': 70589,
 'chaos.social': 64691,
 'hachyderm.io': 58828,
 'mastodon.uno': 54492,
 'social.mikutter.hachune.net': 54279,
 'mastodon.art': 52774,
 'kiritan.work': 47581,
 'planet.moe': 46212,
 'fosstodon.org': 45813,
 'sfba.social': 45382,
 'airwaves.social': 37531,
 'mas.corq.co': 37108,
 'aus.social': 36367,
 'social.tchncs.de': 36132,
 'norden.social': 36090,
 'c

In [10]:
def save_favicon(instance: str, quiet: bool = False) -> Path | None:
    favicon_path = OUTPUT_PATH / "favicons" / f"favicon-{instance}.png"
    favicon_path.parent.mkdir(parents=True, exist_ok=True)
    if favicon_path.exists():
        return favicon_path
    domain = instance
    if instance.count(".") > 1:
        domain = ".".join(instance.split(".")[-2:])
    if not quiet:
        print(f"Searching for favicon for {domain}...")
    try:
        favicons = get_favicon(f"https://{domain}", timeout=2)
    except Exception:
        return None
    favicons = sorted(favicons, key=lambda f: f.width, reverse=True)
    favicons = [favicon for favicon in favicons if favicon.format == "png"]
    if len(favicons) == 0:
        return None
    if not quiet:
        print(f"Downloading favicon from {domain}...")
    favicon_response = get(favicons[0].url, stream=True, timeout=3)
    try:
        with favicon_path.open("wb") as image:
            for chunk in favicon_response.iter_content(1024):
                image.write(chunk)
    except Exception:
        favicon_path.unlink()
        return None
    return favicon_path

In [11]:
top_crawled_instances: list[str] = [
    instance
    for instance, _ in sorted(
        crawled_counts.items(), 
        key=lambda b: b[1],
        reverse=True,
    )[:NUM_TOP_INSTANCES]
]
top_crawled_instances

['mastodon.social',
 'mastodon.online',
 'mstdn.social',
 'ohai.social',
 'mas.to',
 'mastodon.world',
 'universeodon.com',
 'social.vivaldi.net',
 'techhub.social',
 'toot.community']

In [13]:
crawled_instances = set()
with INSTANCES_PATH.open("r") as file:
    crawled_instances.update(line.strip() for line in file)
with REMOVED_INSTANCES_PATH.open("rb") as file:
    crawled_instances.update(load(file))
crawled_instances

{'mastodon.cisti.org',
 'rfj.social',
 'technodon.org',
 'social.darc.de',
 'kurage.cc',
 'protogen.social',
 'rheinneckar.social',
 'blueplanet.social',
 'river.group.lt',
 'archaeo.social',
 'handon.club',
 'chaos.social',
 'social.dino.icu',
 'normal.style',
 'tweesecake.social',
 'restraint.social',
 'mastodon.conquestuniverse.com',
 'gayfr.social',
 'geekdom.social',
 't.chadole.com',
 'ecologie.social',
 'gamedev.lgbt',
 'guitar.rodeo',
 'social.cu2d.com',
 'tooting.ch',
 'yapstreams.com',
 'illo.social',
 'fosstodon.org',
 'econtwitter.net',
 'idlethumbs.social',
 'social.kryta.app',
 'esoteric.party',
 'techhub.social',
 'pogs.cafe',
 'avantwhatever.org',
 'critcare.social',
 'blackrock.city',
 'conversafiada.net',
 'india.goonj.xyz',
 'mastodon.tokyo',
 'recurse.social',
 'gardenstate.social',
 'syringa.social',
 'woofy.social',
 'ohai.social',
 'splat00n.ink',
 'moessingen.social',
 'expressional.social',
 'gaminginthe.cloud',
 'mathtod.online',
 'muenchen.social',
 'mastodon

In [14]:
# "Available" data: the entire Mastodon network (that we know of).
## File to dict
available_local_posts = {}
with INSTANCE_DATA_PATH.open("r") as file:
    for line in file:
        line_dict = loads(line)
        # if line_dict["instance"] not in crawled_instances:
        #     continue
        if "nodeinfo" not in line_dict or line_dict["nodeinfo"] is None:
            continue
        nodeinfo = line_dict["nodeinfo"]
        if "usage" not in nodeinfo or nodeinfo["usage"] is None:
            continue
        usage = nodeinfo["usage"]
        if "localPosts" not in usage or usage["localPosts"] is None:
            continue
        available_local_posts[line_dict["instance"]] = usage["localPosts"]
available_local_posts

{'0rb.it': 860,
 '0ab.uk': 14,
 '0v0.ch': 46,
 '0x4655.co.uk': 0,
 '0xee.io': 199,
 '0x50.ch': 2,
 '10mb.social': 475,
 '100terres.social': 4,
 '0371.uk': 152,
 '0x3c.pl': 2340,
 '1210.nl': 958,
 '103.social': 252,
 '10211104.fedicity.net': 26529,
 '13bells.com': 9489,
 '1435.es': 39,
 '1745.info': 316,
 '15-15-15.social': 392,
 '19marbles.org': 4065,
 '1password.social': 913,
 '184.im': 8,
 '1234.as': 767493,
 '23.illuminati.org': 1569,
 '1uca.com': 219815,
 '285936586.masto.host': 16671,
 '27origin.com': 231,
 '0taku.xyz': 705,
 '2pxnl.de': 1484,
 '280562.com': 115,
 '2c.taoetc.org': 1501,
 '333thats33s.cc': 581,
 '3615.computer': 9511,
 '30d.org': 2,
 '3rdstone.cc': 3178,
 '3dp.chat': 7113,
 '3zi.ru': 135897,
 '3rdwall.net': 36,
 '386898.xyz': 45,
 '366.koyomi.online': 2789,
 '39sounds.net': 2034,
 '41020.social': 7032,
 '338338.xyz': 981,
 '433.world': 12315,
 '4four.org': 97,
 '404.inexist.club': 22468,
 '430022.xyz': 12,
 '4458644.xyz': 24,
 '4bear.com': 101372,
 '503junk.house':

In [15]:
len(available_local_posts)

10817

In [16]:
crawled_data = []
for instance in crawled_counts.keys():
    if instance in top_crawled_instances:
        favicon_path = save_favicon(instance)
    else:
        favicon_path = None
    crawled_data.append({
        "instance": instance,
        "icon": favicon_path if favicon_path is not None else None,
        "posts_available_local": available_local_posts.get(instance, 0),
        "posts_crawled": crawled_counts.get(instance, 0),
        "posts_crawled_federated": crawled_federated_counts.get(instance, 0),
        "posts_crawled_local": crawled_local_counts.get(instance, 0),
    })

df_crawled = DataFrame(crawled_data)
df_crawled["percentage_crawled_federated_of_crawled"] = \
    df_crawled["posts_crawled_federated"] / df_crawled["posts_crawled"]
df_crawled["percentage_crawled_local_of_crawled"] = \
    df_crawled["posts_crawled_local"] / df_crawled["posts_crawled"]
df_crawled["percentage_crawled_of_crawled"] = \
    df_crawled["percentage_crawled_federated_of_crawled"] + \
        df_crawled["percentage_crawled_local_of_crawled"]  # Sanity check.
assert (df_crawled["percentage_crawled_of_crawled"] == 1).all()
df_crawled.drop(columns=["percentage_crawled_of_crawled"])
df_crawled["percentage_crawled_federated_of_all_crawled_federated"] = \
    df_crawled["posts_crawled_federated"] / \
        df_crawled["posts_crawled_federated"].sum()
df_crawled["percentage_crawled_local_of_all_crawled_local"] = \
    df_crawled["posts_crawled_local"] / \
        df_crawled["posts_crawled_local"].sum()
df_crawled["percentage_crawled_of_all_crawled"] = \
    df_crawled["posts_crawled"] / df_crawled["posts_crawled"].sum()
df_crawled["percentage_available_local_of_all_crawled_available_local"] = \
    df_crawled["posts_available_local"] / \
        df_crawled["posts_available_local"].sum()
df_crawled["percentage_available_local_of_all_discovered_available_local"] = \
    df_crawled["posts_available_local"] / \
        sum(available_local_posts.values())
df_crawled["percentage_crawled_local_of_available_local"] = \
    df_crawled["posts_crawled_local"] / df_crawled["posts_available_local"]
df_crawled.head(n=10).style\
    .format({
        col: "{:,.2%}"
        for col in df_crawled.columns
        if col.startswith("percentage_")
    })

,instance,icon,posts_available_local,posts_crawled,posts_crawled_federated,posts_crawled_local,percentage_crawled_federated_of_crawled,percentage_crawled_local_of_crawled,percentage_crawled_of_crawled,percentage_crawled_federated_of_all_crawled_federated,percentage_crawled_local_of_all_crawled_local,percentage_crawled_of_all_crawled,percentage_available_local_of_all_crawled_available_local,percentage_available_local_of_all_discovered_available_local,percentage_crawled_local_of_available_local
0,mastodon.social,../data/favicons/favicon-mastodon.social.png,81205325,15668157,12542364,3125793,80.05%,19.95%,100.00%,1.73%,31.34%,2.14%,21.05%,9.86%,3.85%
1,mastodon.online,../data/favicons/favicon-mastodon.online.png,7759995,9857093,9570010,287083,97.09%,2.91%,100.00%,1.32%,2.88%,1.34%,2.01%,0.94%,3.70%
2,mstdn.social,../data/favicons/favicon-mstdn.social.png,14989224,9638803,9267685,371118,96.15%,3.85%,100.00%,1.28%,3.72%,1.31%,3.89%,1.82%,2.48%
3,ohai.social,../data/favicons/favicon-ohai.social.png,1180837,8754882,8735250,19632,99.78%,0.22%,100.00%,1.21%,0.20%,1.19%,0.31%,0.14%,1.66%
4,mas.to,../data/favicons/favicon-mas.to.png,7021649,8208704,8034084,174620,97.87%,2.13%,100.00%,1.11%,1.75%,1.12%,1.82%,0.85%,2.49%
5,mastodon.world,../data/favicons/favicon-mastodon.world.png,4912465,8149459,7968251,181208,97.78%,2.22%,100.00%,1.10%,1.82%,1.11%,1.27%,0.60%,3.69%
6,universeodon.com,../data/favicons/favicon-universeodon.com.png,2852641,7489453,7399674,89779,98.80%,1.20%,100.00%,1.02%,0.90%,1.02%,0.74%,0.35%,3.15%
7,social.vivaldi.net,../data/favicons/favicon-social.vivaldi.net.png,1709786,7194763,7055731,139032,98.07%,1.93%,100.00%,0.98%,1.39%,0.98%,0.44%,0.21%,8.13%
8,techhub.social,../data/favicons/favicon-techhub.social.png,1404970,7121898,7047603,74295,98.96%,1.04%,100.00%,0.97%,0.74%,0.97%,0.36%,0.17%,5.29%
9,toot.community,../data/favicons/favicon-toot.community.png,1303738,6660323,6630708,29615,99.56%,0.44%,100.00%,0.92%,0.30%,0.91%,0.34%,0.16%,2.27%


In [17]:
print(r"\begin{tabular}{@{}cl@{\hspace{2em}}rr@{\hspace{1.5em}}rrr@{\hspace{1.5em}}rrrr@{\hspace{2em}}rr@{}}")
print(r"  \toprule")
cols = [
    r"\multicolumn{2}{@{}l}{\textbf{Instance}}",
    r"\multicolumn{9}{c}{\textbf{\iconCrawled~Crawled posts}}",
    r"\multicolumn{2}{c}{\textbf{\iconDiscovered~Available}}",
]
print(r"  " + " & ".join(cols) + r" \\")
print(r"  \cmidrule(r{2em}){3-11}\cmidrule{12-13}")
cols = [
    r"",
    r"",
    r"\multicolumn{1}{c}{\iconFederated~Fed.}",
    r"\multicolumn{1}{c@{\hspace{1.5em}}}{{\tiny\faIcon{arrow-down}}\iconFederated}",
    r"\multicolumn{1}{c}{\iconRemote~Remote}",
    r"\multicolumn{1}{c}{{\tiny\faIcon{arrow-down}}\iconRemote}",
    r"\multicolumn{1}{c@{\hspace{1.5em}}}{{\tiny\faIcon{arrow-left}}\iconFederated}",
    r"\multicolumn{1}{c}{\iconLocal~Local}",
    r"\multicolumn{1}{c}{{\tiny\faIcon{arrow-down}}\iconLocal}",
    r"\multicolumn{1}{c}{{\tiny\faIcon{arrow-left}}\iconFederated}",
    r"\multicolumn{1}{c@{\hspace{2em}}}{{\tiny\faIcon{arrow-right}}\iconDiscovered}",
    r"\multicolumn{1}{l}{\iconLocal~Local}",
    r"\multicolumn{1}{c}{{\tiny\faIcon{arrow-down}}\iconCrawled}",
]
print(r"  " + " & ".join(cols) + r" \\")
print(r"  \midrule")
df_crawled_head = df_crawled.sort_values("posts_crawled", ascending=False).head(n=10)
for _, row in df_crawled_head.iterrows():
    favicon_path = save_favicon(row["instance"], quiet=True)
    cols = [
        r"\favicon{" + (row["instance"] if favicon_path is not None else "mastodon.social") + r"}",
        r"\instance{" + row["instance"] + r"}",
        f"{row['posts_crawled']:,d}",
        f"{row['percentage_crawled_of_all_crawled']:,.0%}".replace("%", r"\%"),
        f"{row['posts_crawled_federated']:,d}",
        f"{row['percentage_crawled_federated_of_all_crawled_federated']:,.0%}".replace("%", r"\%"),
        f"{row['percentage_crawled_federated_of_crawled']:,.0%}".replace("%", r"\%"),
        f"{row['posts_crawled_local']:,d}",
        f"{row['percentage_crawled_local_of_all_crawled_local']:,.0%}".replace("%", r"\%"),
        f"{row['percentage_crawled_local_of_crawled']:,.0%}".replace("%", r"\%"),
        f"{row['percentage_crawled_local_of_available_local']:,.0%}".replace("%", r"\%"),
        f"{row['posts_available_local']:,d}",
        f"{row['percentage_available_local_of_all_crawled_available_local']:,.0%}".replace("%", r"\%"),
    ]
    print(r"  " + " & ".join(cols) + r" \\")
df_crawled_tail = df_crawled[~df_crawled.index.isin(df_crawled_head.index)]
cols = [
    r"\iconMore",
    f"{len(df_crawled_tail):,d}" + r" others",
    f"{df_crawled_tail['posts_crawled'].sum():,d}",
    f"{df_crawled_tail['percentage_crawled_of_all_crawled'].sum():,.0%}".replace("%", r"\%"),
    f"{df_crawled_tail['posts_crawled_federated'].sum():,d}",
    f"{df_crawled_tail['percentage_crawled_federated_of_all_crawled_federated'].sum():,.0%}".replace("%", r"\%"),
    f"{(df_crawled_tail['posts_crawled_federated'].sum() / df_crawled['posts_crawled'].sum()):,.0%}".replace("%", r"\%"),
    f"{df_crawled_tail['posts_crawled_local'].sum():,d}",
    f"{df_crawled_tail['percentage_crawled_local_of_all_crawled_local'].sum():,.0%}".replace("%", r"\%"),
    f"{(df_crawled_tail['posts_crawled_local'].sum() / df_crawled['posts_crawled'].sum()):,.0%}".replace("%", r"\%"),
    f"{(df_crawled_tail['posts_crawled_local'].sum() / df_crawled['posts_available_local'].sum()):,.0%}".replace("%", r"\%"),
    f"{df_crawled_tail['posts_available_local'].sum():,d}",
    f"{(df_crawled_tail['percentage_available_local_of_all_crawled_available_local'].sum()):,.0%}".replace("%", r"\%"),
]
cols = [r"\secondary{" + col + r"}" for col in cols]
print(r"  " + " & ".join(cols) + r" \\")
print(r"  \midrule")
cols = [
    r"\iconCrawled",
    f"{len(df_crawled):,d}" + r" crawled",
    f"{df_crawled['posts_crawled'].sum():,d}",
    f"{df_crawled['percentage_crawled_of_all_crawled'].sum():,.0%}".replace("%", r"\%"),
    f"{df_crawled['posts_crawled_federated'].sum():,d}",
    f"{df_crawled['percentage_crawled_federated_of_all_crawled_federated'].sum():,.0%}".replace("%", r"\%"),
    f"{(df_crawled_tail['posts_crawled_federated'].sum() / df_crawled['posts_crawled'].sum()):,.0%}".replace("%", r"\%"),
    f"{df_crawled['posts_crawled_local'].sum():,d}",
    f"{df_crawled['percentage_crawled_local_of_all_crawled_local'].sum():,.0%}".replace("%", r"\%"),
    f"{(df_crawled_tail['posts_crawled_local'].sum() / df_crawled['posts_crawled'].sum()):,.0%}".replace("%", r"\%"),
    f"{(df_crawled['posts_crawled_local'].sum() / df_crawled['posts_available_local'].sum()):,.0%}".replace("%", r"\%"),
    f"{df_crawled['posts_available_local'].sum():,d}",
    f"{(df_crawled['percentage_available_local_of_all_crawled_available_local'].sum()):,.0%}".replace("%", r"\%"),
]
print(r"  " + " & ".join(cols) + r" \\")
cols = [
    r"\iconDiscovered",
    r"10,354~discovered",
    f"---",
    f"---",
    f"---",
    f"---",
    f"---",
    f"---",
    f"---",
    f"---",
    f"---",
    f"{sum(available_local_posts.values()):,d}",
    f"{(sum(available_local_posts.values()) / df_crawled['posts_available_local'].sum()):,.0%}".replace("%", r"\%"),
]
cols = [r"\secondary{" + col + r"}" for col in cols]
print(r"  " + " & ".join(cols) + r" \\")
print(r"  \bottomrule")
print(r"\end{tabular}")

\begin{tabular}{@{}cl@{\hspace{2em}}rr@{\hspace{1.5em}}rrr@{\hspace{1.5em}}rrrr@{\hspace{2em}}rr@{}}
  \toprule
  \multicolumn{2}{@{}l}{\textbf{Instance}} & \multicolumn{9}{c}{\textbf{\iconCrawled~Crawled posts}} & \multicolumn{2}{c}{\textbf{\iconDiscovered~Available}} \\
  \cmidrule(r{2em}){3-11}\cmidrule{12-13}
   &  & \multicolumn{1}{c}{\iconFederated~Fed.} & \multicolumn{1}{c@{\hspace{1.5em}}}{{\tiny\faIcon{arrow-down}}\iconFederated} & \multicolumn{1}{c}{\iconRemote~Remote} & \multicolumn{1}{c}{{\tiny\faIcon{arrow-down}}\iconRemote} & \multicolumn{1}{c@{\hspace{1.5em}}}{{\tiny\faIcon{arrow-left}}\iconFederated} & \multicolumn{1}{c}{\iconLocal~Local} & \multicolumn{1}{c}{{\tiny\faIcon{arrow-down}}\iconLocal} & \multicolumn{1}{c}{{\tiny\faIcon{arrow-left}}\iconFederated} & \multicolumn{1}{c@{\hspace{2em}}}{{\tiny\faIcon{arrow-right}}\iconDiscovered} & \multicolumn{1}{l}{\iconLocal~Local} & \multicolumn{1}{c}{{\tiny\faIcon{arrow-down}}\iconCrawled} \\
  \midrule
  \favicon{mastodon.s